# Advanced Trajectory and Path Planning

## Introduction

Trajectory and path planning are critical components in robotics, autonomous vehicles, and animation. They involve finding a feasible path from a starting point to a goal while satisfying constraints such as obstacle avoidance, kinematics, and dynamics. Advanced methods have been developed to address the complexities of real-world environments.

In this tutorial, we'll delve into advanced methods of trajectory and path planning, exploring the underlying mathematics, implementing example code, and explaining the processes. We'll reference key papers and discuss the latest developments in the field. Relevant imagery generated through code will be included to enhance understanding.

## Table of Contents

1. [Overview of Path and Trajectory Planning](#1)
2. [Sampling-Based Methods](#2)
   - [Rapidly-Exploring Random Trees (RRT)](#2.1)
   - [RRT* Algorithm](#2.2)
   - [Implementation of RRT and RRT*](#2.3)
3. [Optimization-Based Methods](#3)
   - [Trajectory Optimization](#3.1)
   - [Sequential Quadratic Programming (SQP)](#3.2)
   - [Implementation of Trajectory Optimization](#3.3)
4. [Kinodynamic Planning](#4)
   - [Differential Constraints](#4.1)
   - [Implementation with RRT](#4.2)
5. [Model Predictive Control (MPC)](#5)
   - [Formulation](#5.1)
   - [Implementation of MPC](#5.2)
6. [Latest Developments](#6)
   - [Machine Learning in Path Planning](#6.1)
   - [Reinforcement Learning Approaches](#6.2)
7. [Conclusion](#7)
8. [References](#8)

<a id="1"></a>
# 1. Overview of Path and Trajectory Planning

Path planning involves finding a collision-free path from a start point to a goal point in an environment with obstacles. Trajectory planning extends this by considering time, ensuring the path is feasible with respect to the dynamics and kinematics of the system.

### Key Concepts

- **Configuration Space (C-space)**: The space of all possible positions and orientations of the robot.
- **State Space**: Includes both the configuration and its derivatives (e.g., velocities).
- **Obstacle Avoidance**: Ensuring the path does not intersect with obstacles in the environment.
- **Optimality**: Finding the shortest or most efficient path according to a defined cost function.

<a id="2"></a>
# 2. Sampling-Based Methods

Sampling-based algorithms are widely used in path planning due to their ability to handle high-dimensional spaces and complex environments without explicitly constructing the configuration space.

<a id="2.1"></a>
## 2.1 Rapidly-Exploring Random Trees (RRT)

The RRT algorithm [[1]](#ref1) incrementally builds a space-filling tree by randomly sampling the configuration space.

### Algorithm Steps

1. **Initialization**: Start with an initial node at the starting position.
2. **Sampling**: Randomly sample a point $( q_{	ext{rand}} )$ in the configuration space.
3. **Nearest Neighbor**: Find the nearest node $( q_{	ext{near}} )$ in the tree to $( q_{	ext{rand}} )$.
4. **Steer**: Move from $( q_{	ext{near}} )$ towards $( q_{	ext{rand}} )$ by a fixed step size $( \epsilon )$, obtaining $( q_{\text{new}} )$.
5. **Collision Check**: If the path from $( q_{\text{near}} )$ to $( q_{\text{new}} )$ is obstacle-free, add $( q_{\text{new}} )$ to the tree.
6. **Goal Check**: If $( q_{\text{new}} )$ is close to the goal, terminate.

### Mathematical Formulation

- **Nearest Neighbor**:

  $[
  q_{\text{near}} = \operatorname{argmin}_{q \in T} \| q - q_{\text{rand}} |
  ]$

- **Steering Function**:

  $[
  q_{	ext{new}} = q_{\text{near}} + \epsilon \cdot \frac{q_{	ext{rand}} - q_{\text{near}}}{| q_{\text{rand}} - q_{\text{near}} |}
  ]$

<a id="2.2"></a>
## 2.2 RRT* Algorithm

RRT* [[2]](#ref2) is an extension of RRT that provides asymptotic optimality, meaning it converges to the optimal path over time.

### Modifications from RRT

- **Rewiring**: After adding $( q_{\text{new}} )$, check nearby nodes to see if they can be reached more efficiently via $( q_{\text{new}} )$.
- **Cost Function**: Maintain the cost-to-come for each node.

### Algorithm Steps

1. **Same as RRT Steps 1-5**.
2. **Near Nodes**: Find all nodes within a radius $( r )$ of $( q_{\text{new}} )$.
3. **Choose Parent**: Select the node $( q_{\text{min}} )$ that minimizes the cost to $( q_{\text{new}} )$.
4. **Rewire**: For each node in the near set, see if reaching it via $( q_{\text{new}} )$ reduces the cost.

### Mathematical Formulation

- **Cost Function**:

  $[
  c(q) = c(q_{\text{parent}}) + \text{Cost}(q_{\text{parent}}, q)
  ]$

- **Near Nodes Radius**:

  $[
  r = \min\left\{ \gamma \left( \frac{\log n}{n} \right)^{1/d}, \eta \right\}
  ]$

  Where:

  - $( n )$: Number of nodes in the tree.
  - $( d )$: Dimension of the space.
  - $( \gamma )$: A constant.

<a id="2.3"></a>
## 2.3 Implementation of RRT and RRT*

We'll implement RRT and RRT* algorithms in a 2D environment with obstacles.

In [ ]:
# Import necessary libraries
import numpy as np
import matplotlib.pyplot as plt
from matplotlib.patches import Rectangle
%matplotlib inline

In [ ]:
# Define the Node class
class Node:
    def __init__(self, position):
        self.position = position
        self.parent = None
        self.cost = 0.0


In [ ]:
# Define the RRT algorithm
import random

class RRT:
    def __init__(self, start, goal, obstacles, x_limit, y_limit, max_iter=500, step_size=1.0):
        self.start = Node(start)
        self.goal = Node(goal)
        self.obstacles = obstacles
        self.x_limit = x_limit
        self.y_limit = y_limit
        self.max_iter = max_iter
        self.step_size = step_size
        self.nodes = [self.start]
    
    def get_random_point(self):
        x = random.uniform(self.x_limit[0], self.x_limit[1])
        y = random.uniform(self.y_limit[0], self.y_limit[1])
        return np.array([x, y])
    
    def nearest_neighbor(self, q_rand):
        distances = [np.linalg.norm(q_rand - node.position) for node in self.nodes]
        min_index = distances.index(min(distances))
        return self.nodes[min_index]
    
    def steer(self, q_near, q_rand):
        direction = q_rand - q_near.position
        length = np.linalg.norm(direction)
        if length > self.step_size:
            direction = direction / length * self.step_size
        q_new = q_near.position + direction
        return q_new
    
    def collision_free(self, q_near, q_new):
        for obs in self.obstacles:
            if self.check_collision(q_near.position, q_new, obs):
                return False
        return True
    
    def check_collision(self, p1, p2, obs):
        # Simple rectangle collision checking
        x_min, y_min, width, height = obs
        x_max = x_min + width
        y_max = y_min + height
        
        # Check if line segment intersects with rectangle
        # For simplicity, we'll discretize the line segment
        num_steps = int(np.linalg.norm(p2 - p1) / 0.1)
        x_vals = np.linspace(p1[0], p2[0], num_steps)
        y_vals = np.linspace(p1[1], p2[1], num_steps)
        for x, y in zip(x_vals, y_vals):
            if x_min <= x <= x_max and y_min <= y <= y_max:
                return True
        return False
    
    def build_rrt(self):
        for i in range(self.max_iter):
            q_rand = self.get_random_point()
            q_near = self.nearest_neighbor(q_rand)
            q_new = self.steer(q_near, q_rand)
            new_node = Node(q_new)
            if self.collision_free(q_near, new_node.position):
                new_node.parent = q_near
                self.nodes.append(new_node)
                if np.linalg.norm(new_node.position - self.goal.position) < self.step_size:
                    print("Goal reached!")
                    return self.generate_path(new_node)
        print("Goal not reached within max iterations.")
        return None
    
    def generate_path(self, node):
        path = [node.position]
        while node.parent is not None:
            node = node.parent
            path.append(node.position)
        return path[::-1]


In [ ]:
# Define the environment
start = np.array([0, 0])
goal = np.array([10, 10])
obstacles = [
    (3, 3, 2, 6),  # (x_min, y_min, width, height)
    (6, 0, 2, 6)
]
x_limit = (0, 15)
y_limit = (0, 15)

In [ ]:
# Run the RRT algorithm
rrt = RRT(start, goal, obstacles, x_limit, y_limit, max_iter=1000, step_size=0.5)
path = rrt.build_rrt()

In [ ]:
# Visualize the result
def plot_rrt(rrt, path):
    fig, ax = plt.subplots(figsize=(8,8))
    
    # Plot obstacles
    for obs in rrt.obstacles:
        rect = Rectangle((obs[0], obs[1]), obs[2], obs[3], color='gray')
        ax.add_patch(rect)
    
    # Plot nodes and edges
    for node in rrt.nodes:
        if node.parent is not None:
            x_vals = [node.position[0], node.parent.position[0]]
            y_vals = [node.position[1], node.parent.position[1]]
            ax.plot(x_vals, y_vals, color='blue')
    
    # Plot path
    if path is not None:
        path = np.array(path)
        ax.plot(path[:,0], path[:,1], color='red', linewidth=2)
    
    # Plot start and goal
    ax.plot(rrt.start.position[0], rrt.start.position[1], 'go', markersize=10)
    ax.plot(rrt.goal.position[0], rrt.goal.position[1], 'ro', markersize=10)
    
    ax.set_xlim(rrt.x_limit)
    ax.set_ylim(rrt.y_limit)
    ax.set_xlabel('X')
    ax.set_ylabel('Y')
    ax.set_title('RRT Path Planning')
    plt.show()

plot_rrt(rrt, path)

**Explanation:**

- **Node Class**: Represents a point in the space with position, parent, and cost.
- **RRT Class**: Implements the RRT algorithm with methods for sampling, nearest neighbor search, steering, collision checking, and building the tree.
- **Visualization**: Uses matplotlib to display the obstacles, tree, and final path.

**Note:** Implementing RRT* involves additional steps for rewiring. Due to complexity, the full implementation is beyond this example's scope.

<a id="3"></a>
# 3. Optimization-Based Methods

Optimization-based methods formulate the path planning problem as an optimization problem, minimizing a cost function subject to constraints.

<a id="3.1"></a>
## 3.1 Trajectory Optimization

Trajectory optimization involves finding a trajectory that minimizes a cost function while satisfying dynamics and constraints.

### Problem Formulation

Minimize the cost function:

$[
J = \int_{t_0}^{t_f} L(x(t), u(t), t) dt + \phi(x(t_f))
]$

Subject to:

- **Dynamics**:

  $[
  \dot{x}(t) = f(x(t), u(t), t)
  ]$

- **Constraints**:

  $[
  h(x(t), u(t), t) \leq 0
  ]$

### Methods

- **Direct Methods**: Discretize the trajectory and solve the optimization problem.
- **Indirect Methods**: Use calculus of variations and solve the resulting boundary value problem.

<a id="3.2"></a>
## 3.2 Sequential Quadratic Programming (SQP)

SQP is an iterative method for nonlinear optimization problems.

### Algorithm Steps

1. **Initialization**: Start with an initial guess.
2. **Quadratic Approximation**: At each iteration, approximate the problem by a quadratic programming (QP) subproblem.
3. **Solve QP Subproblem**: Solve the QP to find a search direction.
4. **Line Search**: Perform a line search to determine the step size.
5. **Update**: Update the variables and repeat.

### Mathematical Formulation

At iteration $( k )$, solve the QP:

$$
\begin{align*}\min_{\Delta x} \quad & \frac{1}{2} \Delta x^T \nabla^2 L(x_k, \lambda_k) \Delta x + \nabla f(x_k)^T\Delta x 
\\
\\
\text{ subject to} \quad & c(x_k) + \nabla c(x_k)^T \Delta x = 0\end{align*}
$$

Where:

- $( L )$: Lagrangian.
- $( \lambda_k )$: Lagrange multipliers.
- $( c(x) )$: Constraints.

<a id="3.3"></a>
## 3.3 Implementation of Trajectory Optimization

We'll implement a simple trajectory optimization using a direct collocation method.

In [ ]:
# Import libraries
from scipy.optimize import minimize


In [ ]:
# Define the dynamics of a double integrator

def dynamics(x, u):
    dxdt = np.array([x[1], u])
    return dxdt


In [ ]:
# Define the cost function

def cost(z, N, dt):
    cost = 0
    for i in range(N):
        u = z[N*2 + i]
        cost += u**2 * dt  # Minimize control effort
    return cost


In [ ]:
# Define constraints

def constraints(z, x0, xf, N, dt):
    cons = []
    for i in range(N):
        x_i = z[2*i:2*(i+1)]
        x_next = z[2*(i+1):2*(i+2)] if i < N-1 else xf
        u_i = z[N*2 + i]
        
        # Discretize dynamics using Euler method
        x_next_est = x_i + dynamics(x_i, u_i) * dt
        
        cons.extend(x_next - x_next_est)
    
    # Initial condition constraint
    cons_init = z[0:2] - x0
    cons.extend(cons_init)
    return cons


In [ ]:
# Set up the optimization problem
N = 50  # Number of time steps
dt = 0.1  # Time step
x0 = np.array([0, 0])  # Initial position and velocity
xf = np.array([10, 0])  # Final position and velocity

# Initial guess
z0 = np.zeros(N*2 + N)  # [x0, v0, x1, v1, ..., u0, u1, ...]

# Bounds on variables
bounds = []
for _ in range(N):
    bounds.extend([(-np.inf, np.inf), (-np.inf, np.inf)])  # x and v bounds
for _ in range(N):
    bounds.append((-1, 1))  # Control input bounds

# Define constraint function
cons = ({'type': 'eq', 'fun': lambda z: constraints(z, x0, xf, N, dt)})

# Perform optimization
res = minimize(lambda z: cost(z, N, dt), z0, bounds=bounds, constraints=cons)


In [ ]:
# Extract solution
z_opt = res.x
x_opt = z_opt[:N*2].reshape(N, 2)
u_opt = z_opt[N*2:]
t = np.linspace(0, N*dt, N)


In [ ]:
# Plot the results
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(t, x_opt[:,0], label='Position')
plt.plot(t, x_opt[:,1], label='Velocity')
plt.xlabel('Time')
plt.ylabel('State')
plt.legend()
plt.title('Optimal Trajectory')

plt.subplot(1,2,2)
plt.step(t, u_opt, where='post')
plt.xlabel('Time')
plt.ylabel('Control Input')
plt.title('Optimal Control Input')
plt.show()


**Explanation:**

- **Dynamics**: We model a double integrator system (e.g., a simple car moving in one dimension).
- **Cost Function**: Minimize the control effort over the trajectory.
- **Constraints**: Enforce the dynamics and boundary conditions.
- **Optimization**: Use `scipy.optimize.minimize` to solve the constrained optimization problem.

<a id="4"></a>
# 4. Kinodynamic Planning

Kinodynamic planning involves planning in both the configuration space and the velocity space, considering the system's dynamics.

<a id="4.1"></a>
## 4.1 Differential Constraints

The planner must account for the system's differential constraints, i.e., its dynamics and kinematics.

### Examples

- **Car-Like Robots**: Nonholonomic constraints prevent certain motions.
- **Quadrotors**: Must consider acceleration and velocity limits.

<a id="4.2"></a>
## 4.2 Implementation with RRT

Kinodynamic RRT extends RRT by incorporating the system's dynamics.

### Modifications

- **State Space**: Include both position and velocity.
- **Steering Function**: Use control inputs to simulate the system's dynamics over time.
- **Edge Costs**: Can include time or control effort.

**Note:** Implementing a full kinodynamic RRT is complex and beyond the scope of this tutorial. Specialized libraries like OMPL (Open Motion Planning Library) provide implementations.

<a id="5"></a>
# 5. Model Predictive Control (MPC)

MPC is an optimization-based control strategy that solves an optimization problem at each time step, considering a finite prediction horizon.

<a id="5.1"></a>
## 5.1 Formulation

At each time step, MPC solves:

$[
\min_{u_0, ..., u_{N-1}} \sum_{k=0}^{N-1} L(x_k, u_k) + L_f(x_N)
]$

Subject to:

- **Dynamics**: $( x_{k+1} = f(x_k, u_k) )$
- **Constraints**: $( h(x_k, u_k) \leq 0 )$
- **Initial Condition**: $( x_0 = x(t) )$

Only the first control input $( u_0 )$ is applied before the optimization is repeated at the next time step.

<a id="5.2"></a>
## 5.2 Implementation of MPC

We'll implement a simple MPC controller for the double integrator system.

In [ ]:
# MPC parameters
N = 10  # Prediction horizon
dt = 0.1
x_target = np.array([10, 0])

# MPC cost function
def mpc_cost(u_seq, x_current):
    cost = 0
    x = x_current.copy()
    u_seq = u_seq.reshape(N, 1)
    for i in range(N):
        u = u_seq[i]
        x = x + dynamics(x, u) * dt
        cost += np.sum((x - x_target)**2) + u**2
    return cost


In [ ]:
# Simulation parameters
x_current = np.array([0, 0])
T = 20  # Total simulation time
num_steps = int(T / dt)
x_history = []
u_history = []

for _ in range(num_steps):
    # Initial guess for control inputs
    u0 = np.zeros(N)
    
    # Optimize control sequence
    res = minimize(lambda u: mpc_cost(u, x_current), u0, bounds=[(-1, 1)]*N)
    u_opt = res.x
    
    # Apply the first control input
    u_current = u_opt[0]
    x_current = x_current + dynamics(x_current, u_current) * dt
    
    # Save history
    x_history.append(x_current.copy())
    u_history.append(u_current)

x_history = np.array(x_history)
u_history = np.array(u_history)
t = np.linspace(0, T, num_steps)


In [ ]:
# Plot the results
plt.figure(figsize=(12,5))
plt.subplot(1,2,1)
plt.plot(t, x_history[:,0], label='Position')
plt.plot(t, x_history[:,1], label='Velocity')
plt.xlabel('Time')
plt.ylabel('State')
plt.legend()
plt.title('MPC Trajectory')

plt.subplot(1,2,2)
plt.step(t, u_history, where='post')
plt.xlabel('Time')
plt.ylabel('Control Input')
plt.title('MPC Control Input')
plt.show()


**Explanation:**

- **Cost Function**: Minimize the sum of squared differences from the target state and control effort.
- **Constraints**: Control inputs are bounded between -1 and 1.
- **Optimization**: At each time step, solve for the optimal control sequence over the prediction horizon.

<a id="6"></a>
# 6. Latest Developments

Path and trajectory planning continue to evolve with advances in computational power and algorithms.

<a id="6.1"></a>
## 6.1 Machine Learning in Path Planning

Machine learning techniques are being integrated into path planning to handle complex environments.

### Learning-Based Methods

- **Imitation Learning**: Learning from expert demonstrations.
- **Deep Learning**: Using neural networks to approximate cost functions or policies.

### Example

- **Neural RRT** [[3]](#ref3): Uses neural networks to guide the sampling process in RRT.

<a id="6.2"></a>
## 6.2 Reinforcement Learning Approaches

Reinforcement learning (RL) can be used for path planning by learning policies that map observations to actions.

### Advantages

- **Adaptability**: Can handle dynamic and uncertain environments.
- **End-to-End Learning**: From raw sensor data to control inputs.

### Challenges

- **Sample Efficiency**: RL often requires a large number of interactions.
- **Safety**: Ensuring safety during exploration.

### Example

- **Deep Deterministic Policy Gradient (DDPG)**: An RL algorithm suitable for continuous action spaces.

<a id="7"></a>
# 7. Conclusion

Advanced trajectory and path planning methods are essential for autonomous systems operating in complex environments. Sampling-based methods like RRT and RRT* provide probabilistic completeness and, in the case of RRT*, asymptotic optimality. Optimization-based methods offer precise control over constraints and cost functions, while kinodynamic planning incorporates system dynamics. MPC provides a real-time control strategy by solving optimization problems at each time step. The integration of machine learning and reinforcement learning opens new possibilities for adaptive and intelligent planning.

Understanding these methods' underlying mathematics and implementation details equips practitioners to develop robust and efficient path planning algorithms.

<a id="8"></a>
# 8. References

1. <a id="ref1"></a>LaValle, S. M., & Kuffner, J. J. (2001). *Rapidly-Exploring Random Trees: Progress and Prospects*. Algorithmic and Computational Robotics: New Directions, 293–308.
2. <a id="ref2"></a>Karaman, S., & Frazzoli, E. (2011). *Sampling-based Algorithms for Optimal Motion Planning*. The International Journal of Robotics Research, 30(7), 846–894.
3. <a id="ref3"></a>Ichter, B., Harrison, J., & Pavone, M. (2018). *Learning Sampling Distributions for Robot Motion Planning*. 2018 IEEE International Conference on Robotics and Automation (ICRA), 7087–7094.

---

This notebook provides an in-depth exploration of advanced trajectory and path planning methods. You can run the code cells to see how the algorithms are implemented and experiment with different parameters and environments.